# Color image montage: MACS0647 high-z galaxies

Keduse Worku et al.

https://github.com/cosmic-spring/MACS0647-high-z

## Imports

In [1]:
import numpy as np
import string
from glob import glob
import os
from os.path import join
from os.path import expanduser
home = expanduser("~")

In [5]:
# color images
import PIL
from PIL import Image, ImageDraw, ImageFont
PIL.Image.MAX_IMAGE_PIXELS = 933120000  # allow to load large images avoiding DecompressionBombError

In [2]:
import astropy
from astropy.coordinates import SkyCoord

# Load color image

In [30]:
# Color image takes a few seconds to load
field = 'macs0647'
color_image_file = '../v7/color/%s_color.png' % field  # full color image
im = Image.open(color_image_file)
#im = im.transpose(method=Image.FLIP_TOP_BOTTOM)
nx, ny = im.size
im.size

('../v7/color/macs0647_color.png', (20480, 28672, 3))

In [40]:
# convert to numpy array
#color_image = np.asarray(im)
#color_image = color_image[::-1]  # flip top-bottom
#color_image_file, color_image.shape

# Load catalogs

In [76]:
outfile = field + '_phot-eazy.ecsv'
indir = ''  # '../catalogs'
outfile = os.path.join(indir, outfile)
print('Loading', outfile)
v4_catalog = astropy.io.ascii.read(outfile)
v4_catalog[:2]  # show a few rows

Loading macs0647_phot-eazy.ecsv


id,ra,dec,x,y,z_spec,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f115w_flux,f115w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f150w_flux,f150w_fluxerr,f160w_flux,f160w_fluxerr,f200w_flux,f200w_fluxerr,f275wu_flux,f275wu_fluxerr,f277w_flux,f277w_fluxerr,f336wu_flux,f336wu_fluxerr,f356w_flux,f356w_fluxerr,f390wu_flux,f390wu_fluxerr,f435w_flux,f435w_fluxerr,f444w_flux,f444w_fluxerr,f475w_flux,f475w_fluxerr,f555w_flux,f555w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr,nusefilt,z_ml,z_ml_chi2,z_ml_risk,lc_min,lc_max,z_phot,z_phot_chi2,z_phot_risk,z_min_risk,min_risk,z_raw_chi2,raw_chi2,z025,z160,z500,z840,z975,restU,restU_err,restB,restB_err,restV,restV_err,restJ,restJ_err,dL,mass,sfr,Lv,LIR,energy_abs,Lu,Lj,L1400,L2800,LHa,LOIII,LHb,LOII,MLv,Av,lwAgeV,mass_p,sfr_p,Lv_p,LIR_p,energy_abs_p,Lu_p,Lj_p,L1400_p,L2800_p,LHa_p,LOIII_p,LHb_p,LOII_p,Av_p,ssfr_p,rest120,rest120_err,rest121,rest121_err,rest156,rest156_err,rest157,rest157_err,rest158,rest158_err,rest159,rest159_err,rest160,rest160_err,rest414,rest414_err,rest415,rest415_err,rest416,rest416_err
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Mpc,solMass,solMass / yr,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solMass / solLum,,Gyr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int32,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.015220,70.161446,4098.274752800464,1.999824071826548,-1.0,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0,-1.0,-1.0,-1.0,27082581599519.1,0.0,-1.0,-1.0,-1.0,0.01,0.0,0.01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,0.0e+00,-9.00e+29,-900000000000000046043660025856.000,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.000 .. -900000000000000046043660025856.000,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.00 .. -900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000
2,101.948789,70.161459,8156.305414383205,1.9979236794378306,-1.0,-99.000,-99.000,-99.000,-99.

In [15]:
indir = '../v7/catalogs'
outfile = os.path.join(indir, outfile)
print('Loading', outfile)
v7_catalog = astropy.io.ascii.read(outfile)
v7_catalog[:2]

Loading ../v7/catalogs/macs0647_phot-eazy.ecsv


id,ra,dec,x,y,z_spec,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f115w_flux,f115w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f150w_flux,f150w_fluxerr,f160w_flux,f160w_fluxerr,f200w_flux,f200w_fluxerr,f275wu_flux,f275wu_fluxerr,f277w_flux,f277w_fluxerr,f336wu_flux,f336wu_fluxerr,f350lpu_flux,f350lpu_fluxerr,f356w_flux,f356w_fluxerr,f390wu_flux,f390wu_fluxerr,f435w_flux,f435w_fluxerr,f444w_flux,f444w_fluxerr,f475w_flux,f475w_fluxerr,f480m_flux,f480m_fluxerr,f555w_flux,f555w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr,nusefilt,z_ml,z_ml_chi2,z_ml_risk,lc_min,lc_max,z_phot,z_phot_chi2,z_phot_risk,z_min_risk,min_risk,z_raw_chi2,raw_chi2,z025,z160,z500,z840,z975,restU,restU_err,restB,restB_err,restV,restV_err,restJ,restJ_err,dL,mass,sfr,Lv,LIR,energy_abs,Lu,Lj,L1400,L2800,LHa,LOIII,LHb,LOII,MLv,Av,lwAgeV,mass_p,sfr_p,Lv_p,LIR_p,energy_abs_p,Lu_p,Lj_p,L1400_p,L2800_p,LHa_p,LOIII_p,LHb_p,LOII_p,Av_p,ssfr_p,rest120,rest120_err,rest121,rest121_err,rest156,rest156_err,rest157,rest157_err,rest158,rest158_err,rest159,rest159_err,rest160,rest160_err,rest414,rest414_err,rest415,rest415_err,rest416,rest416_err
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Mpc,solMass,solMass / yr,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solMass / solLum,,Gyr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int32,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.168830,70.172701,866.1075251495347,1.9806854281628345,-1.0,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0,-1.0,-1.0,-1.0,27082581599519.1,0.0,-1.0,-1.0,-1.0,0.01,0.0,0.01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0e+00,-9.00e+29,-900000000000000046043660025856.000,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.000 .. -900000000000000046043660025856.000,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.00 .. -900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00

In [18]:
indir = '../v7/catalogs'
infile = 'highzids_v4_new.txt'
infile = os.path.join(indir, infile)
print('Loading', infile)
highz_v4_ids = np.loadtxt(infile)
highz_v4_ids

Loading ../v7/catalogs/highzids_v4_new.txt


array([2876., 3035., 3090., 3138., 3186., 3208., 3308., 3493., 3504.,
       3568., 3643., 4186., 4219., 4514., 4775., 4838., 4839., 4865.,
       4893., 4914., 4915., 4920., 5010., 5094., 5156., 5332., 5352.,
       5420., 1742., 1657., 1388.,  457.,  343.,  708.,   47., 1579.,
       1663.,  643., 1240., 2173., 1881., 1557., 1674.,  884.,  747.,
       2011., 2330., 1711., 1395.,  636., 1757.,  811., 1727.,  153.,
        844.,  642., 1559.,  569.,  157., 1035.,  246., 2675.,  486.,
        986., 2590., 1924.,  198., 2196., 1243., 1751., 2241.,  475.,
       2562., 2244., 2127., 2642., 2251., 2390.,  753., 2341., 2095.,
       2557., 2094., 2415., 2070., 2134., 1139.,  946., 2019., 1882.,
        926.,  116., 1714., 2634.,  852., 1741., 2162.,  661., 1912.,
       2421., 1378., 1944., 1673.])

In [23]:
v7_catalog_coordinates = SkyCoord(ra=v7_catalog['ra']*u.deg, dec=v7_catalog['dec']*u.deg)  # *u.deg

In [116]:
v4_catalog_coordinates = SkyCoord(ra=v4_catalog['ra']*u.deg, dec=v4_catalog['dec']*u.deg)  # *u.deg

# Make an image stamp for one galaxy

In [83]:
#for v4_id in highz_v4_ids:
v4_id = highz_v4_ids[2]
i_v4 = np.searchsorted(v4_catalog['id'], v4_id)
RA  = v4_catalog['ra'][i_v4]
Dec = v4_catalog['dec'][i_v4]
object_coordinates = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg)
i_v7, match_distance_2d, match_distance_3d = object_coordinates.match_to_catalog_sky(v7_catalog_coordinates)
v7_id = v7_catalog['id'][i_v7]
match_distance_2d.to(u.arcsec)[0]  # good match?
int(v4_id), v7_id

(3090, 4268)

https://relics.stsci.edu/data/macs0647/JWST/EAZY/eazy_3090.png  
https://relics.stsci.edu/data/macs0647/JWST/v7/EAZY/eazy_4268.png

In [91]:
x = v7_catalog['x'][i_v7]
y = v7_catalog['y'][i_v7]
y = ny - y  # color image is flipped top to bottom

image stamps 200 pixels = 4" on a side (0.02" / pixel)

In [92]:
dx = dy = 100
im_stamp = im.crop((x-dx, y-dy, x+dx, y+dy))

In [93]:
#im_stamp.show()
draw = ImageDraw.Draw(im_stamp) 

In [94]:
#text  =   'id_v7 #%d' % v7_id
#text += '\nid_v4 #%d' % v4_id

In [95]:
text = '#%d v7, #%d v4' % (v7_id, v4_id)
text += '\n'
text += 'z = %.2f' % v7_catalog['z_phot'][i_v7]
text += ' [%.2f - %.2f]' % (v7_catalog['z025'][i_v7], v7_catalog['z975'][i_v7])
text

'#4268 v7, #3090 v4\nz = 6.29 [6.17 - 6.51]'

In [96]:
text
draw.text((5, 5), text)

In [97]:
im_stamp.show()

In [79]:
v7_catalog['z_phot'][i_v7]
v7_catalog['z025'][i_v7]
v7_catalog['z975'][i_v7]

16.839136

# Making a collage
https://stackoverflow.com/questions/35438802/making-a-collage-in-pil

In [98]:
nobjects = len(highz_v4_ids)
ncolumns = 8
nrows = np.ceil(nobjects / ncolumns).astype(int)
nobjects, ncolumns, nrows

(103, 8, 13)

In [99]:
margin = 5
width  = (2*dx + margin) * ncolumns - margin
height = (2*dy + margin) * nrows    - margin

In [100]:
im_collage = Image.new('RGB', (width, height), 'white')
for i in range(nobjects):
    #for v4_id in highz_v4_ids:
    v4_id = highz_v4_ids[i]
    i_v4 = np.searchsorted(v4_catalog['id'], v4_id)
    RA  = v4_catalog['ra'][i_v4]
    Dec = v4_catalog['dec'][i_v4]
    object_coordinates = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg)
    i_v7, match_distance_2d, match_distance_3d = object_coordinates.match_to_catalog_sky(v7_catalog_coordinates)
    v7_id = v7_catalog['id'][i_v7]
    d2d = match_distance_2d.to(u.arcsec)[0]  # good match?

    x = v7_catalog['x'][i_v7]
    y = v7_catalog['y'][i_v7]
    y = ny - y  # color image is flipped top to bottom
    im_stamp = im.crop((x-dx, y-dy, x+dx, y+dy))
    draw = ImageDraw.Draw(im_stamp) 
    text = '#%d v7, #%d v4' % (v7_id, v4_id)
    text += '\n'
    text += 'z = %.2f' % v7_catalog['z_phot'][i_v7]
    text += ' [%.2f - %.2f]' % (v7_catalog['z025'][i_v7], v7_catalog['z975'][i_v7])
    draw.text((5, 5), text)

    iy = i // ncolumns
    ix = i % ncolumns
    ystamp = iy * (2*dy + margin)
    xstamp = ix * (2*dx + margin)
    im_collage.paste(im_stamp, (xstamp, ystamp))
    
    #print(i, iy, ix, v7_id, v4_id, d2d)

im_collage.save('macs0647_high-z_collage.png')
im_collage.show()

# v7 collage

In [112]:
indir = '../v7/catalogs'
infile = 'highzids_v7_final.txt'
infile = os.path.join(indir, infile)
print('Loading', infile)
highz_v7_ids = np.loadtxt(infile)
highz_v7_ids = np.sort(highz_v7_ids)
highz_v7_ids

Loading ../v7/catalogs/highzids_v7_final.txt


array([ 173.,  274.,  587.,  792.,  904., 1021., 1107., 1133., 1175.,
       1352., 1471., 1626., 1658., 1857., 2120., 2121., 2172., 2193.,
       2426., 2510., 2551., 2619., 2846., 4010., 4213., 4268., 4336.,
       4393., 4411., 4533., 4797., 4824., 4870., 4922., 5091., 5191.,
       5591., 5881., 5931., 6339., 6736., 6839., 6871., 6901., 6941.,
       6943., 6952., 7120., 7282., 7397., 7576., 7604., 7697.])

In [113]:
nobjects = len(highz_v7_ids)
ncolumns = 7
nrows = np.ceil(nobjects / ncolumns).astype(int)

margin = 5
width  = (2*dx + margin) * ncolumns - margin
height = (2*dy + margin) * nrows    - margin

nobjects, ncolumns, nrows

(53, 7, 8)

In [117]:
im_collage = Image.new('RGB', (width, height), 'white')
for i in range(nobjects):
    v7_id = highz_v7_ids[i]
    i_v7 = np.searchsorted(v7_catalog['id'], v7_id)
    RA  = v7_catalog['ra'][i_v7]
    Dec = v7_catalog['dec'][i_v7]
    object_coordinates = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg)
    i_v4, match_distance_2d, match_distance_3d = object_coordinates.match_to_catalog_sky(v4_catalog_coordinates)
    v4_id = v4_catalog['id'][i_v4]
    d2d = match_distance_2d.to(u.arcsec)[0]  # good match?
    
    x = v7_catalog['x'][i_v7]
    y = v7_catalog['y'][i_v7]
    y = ny - y  # color image is flipped top to bottom
    im_stamp = im.crop((x-dx, y-dy, x+dx, y+dy))
    draw = ImageDraw.Draw(im_stamp) 
    text = '#%d v7, #%d v4' % (v7_id, v4_id)
    text += '\n'
    text += 'z = %.2f' % v7_catalog['z_phot'][i_v7]
    text += ' [%.2f - %.2f]' % (v7_catalog['z025'][i_v7], v7_catalog['z975'][i_v7])
    draw.text((5, 5), text)

    iy = i // ncolumns
    ix = i % ncolumns
    ystamp = iy * (2*dy + margin)
    xstamp = ix * (2*dx + margin)
    im_collage.paste(im_stamp, (xstamp, ystamp))
    
    #print(i, iy, ix, v7_id, v4_id, d2d)

im_collage.save('macs0647_high-z_collage.png')
im_collage.show()